In [1]:
# %%capture
# !wget https://www.dropbox.com/scl/fi/3mwyjefpr86kqktwrs1dh/HeadBest.pt?rlkey=b1h3rsr1z4cb82yh2najhvmrj&st=k4l2wefe&dl=0
# !mv HeadBest.pt?rlkey=b1h3rsr1z4cb82yh2najhvmrj HeadBest.pt
     
# %%capture
# !wget https://www.dropbox.com/scl/fi/ejtve3v8cggqo1d7vfmr9/dataSetUnlabel.zip?rlkey=lyun664rceo9shrfie7rh9wsc&st=aldjww59&dl=0
# !mv dataSetUnlabel.zip?rlkey=lyun664rceo9shrfie7rh9wsc dataSetUnlabel.zip
# !unzip dataSetUnlabel.zip    

In [2]:
from ultralytics import YOLO

# Cargar el modelo preentrenado
model = YOLO('HeadBest.pt')

In [3]:
import os
import shutil
import random

# Definir las rutas a los directorios
base_dir = 'dataSet/'  # Reemplaza esto con la ruta correcta
unlabel_dir = os.path.join(base_dir, 'Unlabel')
train_images_dir = os.path.join(base_dir, 'images', 'train')
train_labels_dir = os.path.join(base_dir, 'labels', 'train')
processed_images_file = os.path.join(base_dir, 'processed_images.txt')

# Crear los directorios de destino si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)

def load_processed_images(processed_images_file):
    if os.path.exists(processed_images_file):
        with open(processed_images_file, 'r') as f:
            return set(line.strip() for line in f)
    return set()

def save_processed_images(processed_images_file, processed_images):
    with open(processed_images_file, 'w') as f:
        for image_path in processed_images:
            f.write(f"{image_path}\n")

def get_unlabeled_images(batch_size=100):
    processed_images = load_processed_images(processed_images_file)
    all_images = [os.path.join(unlabel_dir, f) for f in os.listdir(unlabel_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Filtrar las imágenes ya procesadas y verificar existencia
    unprocessed_images = [img for img in all_images if img not in processed_images and os.path.exists(img)]

    # Seleccionar secuencialmente las primeras `batch_size` imágenes no procesadas
    return unprocessed_images[:batch_size]

def predict_and_label_images(images, model, confidence_threshold=0.7):
    labeled_images = {}

    for image_path in images:
        results = model.predict(image_path, conf=confidence_threshold, verbose=False)

        for result in results:
            boxes = result.boxes
            confs = boxes.conf.cpu().numpy()
            clss = boxes.cls.cpu().numpy()
            data = boxes.xywhn.cpu().numpy()

            for conf, box, cls in zip(confs, data, clss):
                if conf >= confidence_threshold:
                    class_id = int(cls)  # Convertir la clase a entero
                    x_center, y_center, width, height = box[:4]  # Coordenadas de la caja
                    if image_path not in labeled_images:
                        labeled_images[image_path] = []
                    labeled_images[image_path].append((class_id, x_center, y_center, width, height))

    return labeled_images

def save_predictions(predicted_labels, train_images_dir, train_labels_dir, processed_images):
    for image_path, predictions in predicted_labels.items():

        # Generar un nuevo nombre de archivo basado en los existentes en la carpeta de entrenamiento
        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(train_images_dir, image_filename)

        if os.path.exists(new_image_path):
            base_name, ext = os.path.splitext(image_filename)
            counter = 1
            new_image_path = os.path.join(train_images_dir, f"{base_name}_{counter}{ext}")
            while os.path.exists(new_image_path):
                counter += 1
                new_image_path = os.path.join(train_images_dir, f"{base_name}_{counter}{ext}")

        # Mover la imagen al directorio de entrenamiento
        shutil.move(image_path, new_image_path)

        # Guardar las etiquetas en el formato YOLO
        label_path = os.path.join(train_labels_dir, f"{os.path.splitext(image_filename)[0]}.txt")
        with open(label_path, 'w') as f:
            for class_id, x_center, y_center, width, height in predictions:
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

        # Añadir la imagen al conjunto de imágenes procesadas
        processed_images.add(image_path)

def pseudolabeling_iteration(model, num_iterations=5, batch_size=100, confidence_threshold=0.7):
    processed_images = load_processed_images(processed_images_file)

    for i in range(num_iterations):
        print(f"Iteración {i+1}/{num_iterations}")

        # Seleccionar un lote de imágenes sin etiquetar
        unlabeled_images = get_unlabeled_images(batch_size)

        if not unlabeled_images:
            print("No quedan más imágenes sin etiquetar.")
            break

        # Obtener las predicciones para el lote de imágenes
        predicted_labels = predict_and_label_images(unlabeled_images, model, confidence_threshold)

        # Guardar las predicciones y mover las imágenes etiquetadas
        save_predictions(predicted_labels, train_images_dir, train_labels_dir, processed_images)

        # Guardar el estado de las imágenes procesadas
        save_processed_images(processed_images_file, processed_images)

        # Entrenar el Modelo
        # Cada época tarda unos 2 minutos. Luego son un máximo de 2000 minutos por entrenamiento. Por 5 veces, se nos va a
        # una semana. Conclusión: demasiado tiempo. Lo bajo a 300 épocas.
        model.train(data='dataSet.yaml',
                      epochs=300,
                      #imgsz=250,
                      batch=-1,
                      plots = True)

# Ejecutar el proceso de pseudolabeling
pseudolabeling_iteration(model, num_iterations=5, batch_size=30000, confidence_threshold=0.7)

Iteración 1/5
New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.9 🚀 Python-3.9.13 torch-1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=HeadBest.pt, data=dataSet.yaml, epochs=300, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnos

train: Scanning /grupob/home/jodivaso/tfm_miguel/pseudolabelling/dataSet/labels/train... 12248 images, 122 backgrounds, 0 corrupt: 100%|██████████| 12370/12370 [00:07<00:00, 1622.50it/s]


train: New cache created: /grupob/home/jodivaso/tfm_miguel/pseudolabelling/dataSet/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /grupob/home/jodivaso/tfm_miguel/pseudolabelling/dataSet/labels/val... 109 images, 0 backgrounds, 0 corrupt: 100%|██████████| 109/109 [00:00<00:00, 1503.49it/s]

val: New cache created: /grupob/home/jodivaso/tfm_miguel/pseudolabelling/dataSet/labels/val.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    156/300      15.5G     0.6334     0.5456      1.113         13        640: 100%|██████████| 318/318 [02:01<00:00,  2.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.51it/s]

                   all        109        224      0.738      0.534      0.635      0.294



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    157/300      15.5G     0.6313     0.5522      1.116         71        640:  50%|█████     | 159/318 [01:01<01:01,  2.60it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:

metrics = model.val(split = 'test', plots = True)
metrics.results_dict

Ultralytics YOLOv8.2.9 🚀 Python-3.9.13 torch-1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /grupob/home/jodivaso/tfm_miguel/pseudolabelling/dataSet/labels/test... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<00:00, 1407.68it/s]

val: New cache created: /grupob/home/jodivaso/tfm_miguel/pseudolabelling/dataSet/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.62it/s]


                   all         42         85      0.754      0.577      0.712      0.332
                  Head         42         42      0.788      0.595      0.741      0.393
                  Tail         42         43      0.719      0.558      0.683      0.272
Speed: 2.8ms preprocess, 11.5ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train422222


{'metrics/precision(B)': 0.7537217446583042,
 'metrics/recall(B)': 0.5766888150609081,
 'metrics/mAP50(B)': 0.7115925468158355,
 'metrics/mAP50-95(B)': 0.33240892311300974,
 'fitness': 0.37032728548329236}

In [5]:

import shutil
shutil.make_archive('images', 'zip',train_images_dir)
shutil.make_archive('labels', 'zip',train_labels_dir)

'/grupob/home/jodivaso/tfm_miguel/pseudolabelling/labels.zip'